![aiedge](https://ai-edge.be/assets/img/ai-edge.fe085446.png)

# XOR MODEL
This notebook will guide you through the process of training and exporting a model from TensorFlow to TFLite Micro.  
Afterwards, we can than run this model on a microcontroller, such as the ESP32.

In order to keep the training times manageable, we will train a very simple model.  
The model will be given two inputs and computes the XOR gate between those two inputs.

<table style="margin-left: 0">
    <thead>
    <tr><th>IN1</th><th>IN2</th><th style="border-left: 1px solid">OUT</th></tr>
    </thead>
    <tbody>
    <tr>
        <td style="text-align: center">0</td>
        <td style="text-align: center">0</td>
        <td style="text-align: center; border-left: 1px solid">0</td>
    </tr>
    <tr>
        <td style="text-align: center">0</td>
        <td style="text-align: center">1</td>
        <td style="text-align: center; border-left: 1px solid">1</td>
    </tr>
    <tr>
        <td style="text-align: center">1</td>
        <td style="text-align: center">0</td>
        <td style="text-align: center; border-left: 1px solid">1</td>
    </tr>
    <tr>
        <td style="text-align: center">1</td>
        <td style="text-align: center">1</td>
        <td style="text-align: center; border-left: 1px solid">0</td>
    </tr>
    </tbody>
</table>

For more information, check out the TensorFlow documentation: https://www.tensorflow.org/lite.

In [1]:
import itertools
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset

print(tf.__version__)

2.4.1


## Training the Model

### Model
Our model takes two binary values as input and should output a binary value which is the XOR gate of the two inputs.  
However, when modeling this as a deep learning problem, we will work with floating point values.  
Our inputs will thus be **0.0** or **1.0**, representing the two valid boolean states.  
Our output will then be a floating point value between **0.0** and **1.0**, which represents the probability of the output being one.

We don't need a very complicated model for our problem, so we'll just define a small neural network with an input layer and an output layer.  
It's important that the activation function for the output should be sigmoid, as we want a final output between **0.0** and **1.0**.

In [2]:
model = Sequential([
    Input(shape=(2)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

For our loss function we need to use a BinaryCrossentropy loss.  
Crossentropy quantifies the difference between two probability distribution.
We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.

In [3]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'],
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 129
Trainable params: 129
Non-trainable params: 0
_________________________________________________________________


### Training Dataset
We'll create a python generator and feed that through a tensorflow Dataset to train our model.  
This dataset endlessly generates data, by getting 2 random booleans and computing the ground truth XOR.

In [4]:
def training_data_generator():
    while(True):
        # Generate 2 random boolean values
        # Note that these inputs are numbers, which is what our model requires
        bool1 = random.getrandbits(1)
        bool2 = random.getrandbits(1)
        data = [bool1, bool2]
        
        # Generate ground truth: bool1 XOR bool2
        # Note that the ground truth should be a list containing a single number (zero or one)
        ground_truth = [bool1 != bool2]
        
        # Yield result
        yield data, ground_truth

You can use the following cell to test your dataset:

In [5]:
# Create dataset
generator = training_data_generator()

# Print 10 elements from the dataset
for _ in range(10):
    data, ground_truth = next(generator)
    print(f'Data: {data}')
    print(f'Ground truth: {ground_truth}')
    print('-----')

Data: [0, 1]
Ground truth: [True]
-----
Data: [1, 1]
Ground truth: [False]
-----
Data: [1, 1]
Ground truth: [False]
-----
Data: [0, 1]
Ground truth: [True]
-----
Data: [0, 1]
Ground truth: [True]
-----
Data: [0, 0]
Ground truth: [False]
-----
Data: [1, 0]
Ground truth: [True]
-----
Data: [0, 1]
Ground truth: [True]
-----
Data: [0, 1]
Ground truth: [True]
-----
Data: [1, 0]
Ground truth: [True]
-----


### Training
We can now train the model.  
You should see the loss decreasing and the accuracy increasing towards **100%**.

In [6]:
# Create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    training_data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1)),
)
train_dataset = train_dataset.batch(batch_size=32)

In [7]:
# Train model
model.fit(
    train_dataset,
    steps_per_epoch=500,
    epochs=4,
)

Epoch 1/4
500/500 [==============================] - 5s 8ms/step - loss: 0.5490 - accuracy: 0.9383
Epoch 2/4
500/500 [==============================] - 4s 9ms/step - loss: 0.1825 - accuracy: 1.0000
Epoch 3/4
500/500 [==============================] - 4s 9ms/step - loss: 0.0597 - accuracy: 1.0000
Epoch 4/4
500/500 [==============================] - 4s 9ms/step - loss: 0.0247 - accuracy: 1.0000


### Testing Inference
It is important to test the model results after training.  
Usually this is done with a testing dataset, but as we have only 4 valid input pairs, we will simply run the model and check these 4 results manually.  
These results can later be checked on the ESP device, in order to assess whether the model was exported successfully.

In [8]:
# Create test inputs
test_X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])

# Run model and convert output to list
predicted_probabilities = model.predict_on_batch(test_X)
predicted_probabilities = predicted_probabilities.squeeze().tolist()

# Loop through the raw values from Y and interpret those results.
# The output should be a boolean (True or False), which is the XOR of the inputs.
predicted_xor = []
for element in predicted_probabilities:
    predicted_xor.append(element > 0.5)

print('RAW OUTPUT:', predicted_probabilities)
print('XOR OUTPUT:', predicted_xor)

RAW OUTPUT: [0.03497999906539917, 0.9934456944465637, 0.9921950101852417, 0.004765838384628296]
XOR OUTPUT: [False, True, True, False]


## Exporting the Model
Now we just need to export the model to TFLite.  
Note that this step allows you to perform some optimizations, which will be discussed later.

### Convert to TFLite
Once we are happy with our **Keras model**, we can convert it to TFLite.  
_You can use the [following guide](https://www.tensorflow.org/lite/convert/index#python_api) to fill in the blanks in the next code section._

In [9]:
# Convert the keras model from tensorflow to TFLite.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save tflite model to a file
with open('xormodel.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpqya79lel/assets


We have now saved our TFLite model to a file, which we can visualize with Netron.  
<a href="http://localhost:1234" target="_blank">Click here</a> to open netron and select your file to visualize it.  
After clicking on "Open Model...", you can press **CTRL+L** and enter the following path: _~/aiedge/xormodel/notebook/xormodel.tflite_

### Convert to C++ data
Finally, we need to convert the TFLite model to a C++ array.

In [10]:
!xxd -i xormodel.tflite > xormodel.cpp
!cat xormodel.cpp

unsigned char xormodel_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x1c, 0x00, 0x14, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00, 0xa4, 0x00, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0x4c, 0x00, 0x00, 0x00, 0x44, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0xa0, 0x02, 0x00, 0x00, 0x40, 0x01, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0xa8, 0x00, 0x00, 0x00, 0x0a, 0x00, 0x00, 0x00,
  0xf8, 0x06, 0x00, 0x00, 0xf4, 0x06, 0x00, 0x00, 0xa4, 0x05, 0x00, 0x00,
  0x2c, 0x05, 0x00, 0x00, 0xdc, 0x03, 0x00, 0x00, 0x08, 0x03, 0x00, 0x00,
  0xe0, 0x06, 0x00, 0x00, 0xdc, 0x06, 0x00, 0x00, 0xd8, 0x06, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x04, 0x00, 0x08, 0x00,
  0x08, 0x00, 0x00, 0x00, 0x08, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00,
  

Congratulations, you have now succesfully:
- Created and trained model using tensorflow and keras
- Exported the model to TFLite
- Converted the binary model to a CPP file, so we can use it in TFLite Micro.

![eavise](https://gitlab.com/EAVISE/branding/logo/-/raw/master/twitter/header_500.png)